In [1]:
import csv

In [2]:
import imageio
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
import os

In [3]:
data_dir = 'data/sample_data/'

### Load from CSV

In [4]:
df = pd.read_csv(data_dir+'driving_log.csv')
df.columns

Index(['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed'], dtype='object')

In [5]:
def load_images(data_dir, valid_cameras):
    df = pd.read_csv(data_dir+'driving_log.csv')
    images = []
    measurements = []
    camera_angle = []

    def fetch_image(data_dir, source_path):
        filename = source_path.split('/')[-1]
        current_path = data_dir + 'IMG/' + filename
        return imageio.imread(current_path)

    def append_image(image, measurement, camera):
        images.append(image)
        measurements.append(measurement)
        camera_angle.append(camera)

        # flip image
        image_flipped = np.fliplr(image)
        measurement_flipped = -measurement
        images.append(image_flipped)
        measurements.append(measurement_flipped)
        camera_angle.append(camera)

    for idx, row in df.iterrows():
        measurement = float(row['steering'])
        camera_map = { 'left':0, 'center':1, 'right':2 }
        for c in valid_cameras:
            image = fetch_image(data_dir, row[c])
            append_image(image, measurement, camera_map[c])
            
    return images, measurements, camera_angle
    
def load_from_dir(data_dir, valid_cameras=['center'], load_cached=True):
    if load_cached and os.path.exists(data_dir+'X_train.npy'):
        X_train = np.load(data_dir+'X_train.npy')
        y_train = np.load(data_dir+'y_train.npy')
        camera_train = np.load(data_dir+'camera_train.npy')
    else:
        images, measurements, camera_angle = load_images(data_dir, valid_cameras)
        
        label_binarizer = LabelBinarizer()
        camera_train = label_binarizer.fit_transform(camera_angle)
        X_train = np.array(images)
        y_train = np.array(measurements)

        np.save(data_dir+'X_train.npy', X_train)
        np.save(data_dir+'y_train.npy', y_train)
        np.save(data_dir+'camera_train.npy', camera_train)
    
    print('Camera:', camera_train.shape)
    print('X_train:', X_train.shape)
    print('y_train:', y_train.shape)
    return X_train, camera_train, y_train
    

In [6]:
X_train, camera_train, y_train = load_from_dir(data_dir, ['left', 'center', 'right'], False)

Camera: (48216, 3)
X_train: (48216, 160, 320, 3)
y_train: (48216,)


### Create Model

In [7]:
import keras
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Convolution2D, BatchNormalization, Input

Using TensorFlow backend.


### Multi Cameras (no metadata)

In [8]:
camera_input = Input((3,))
camera_input_bn = BatchNormalization()(camera_input)

inputs = Input(shape=(160,320,3))
# inputs = Input(shape=(66,200,3)) # Checking with NVidia dimensions
x = Lambda(lambda x: x - 255.0 / 255.0)(inputs)

x = Convolution2D(24, (5,5), strides=(2,2), activation='relu')(x)
x = Convolution2D(36, (5,5), strides=(2,2), activation='relu')(x)
x = Convolution2D(48, (5,5), strides=(2,2), activation='relu')(x)
x = Convolution2D(64, (3,3), strides=(1,1), activation='relu')(x)
x = Convolution2D(64, (3,3), strides=(1,1), activation='relu')(x)
x = Flatten()(x)
x = Dense(100, activation='relu')(x)
x = Dense(50, activation='relu')(x)
x = Dense(10, activation='relu')(x)
x = Dense(1)(x)

model = Model(inputs, x)
model.compile(loss='mse', optimizer='adam')

In [10]:
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=7)

# model.save(data_dir+'nvidia_suggested_model.h5')

Train on 38572 samples, validate on 9644 samples
Epoch 1/7
38572/38572 [==============================] - 42s - loss: 0.7173 - val_loss: 0.0167
Epoch 2/7
38572/38572 [==============================] - 40s - loss: 0.0166 - val_loss: 0.0167
Epoch 3/7
14560/38572 [==========>...................] - ETA: 22s - loss: 0.0161

KeyboardInterrupt: 

### With Camera Metadata

In [13]:
camera_input = Input((3,)) 
camera_input_bn = BatchNormalization()(camera_input)

inputs = Input(shape=(160,320,3))
# inputs = Input(shape=(66,200,3)) # Checking with NVidia dimensions
x = Lambda(lambda x: x - 255.0 / 255.0)(inputs)

x = Convolution2D(24, (5,5), strides=(2,2), activation='relu')(x)
x = Convolution2D(36, (5,5), strides=(2,2), activation='relu')(x)
x = Convolution2D(48, (5,5), strides=(2,2), activation='relu')(x)
x = Convolution2D(64, (3,3), strides=(1,1), activation='relu')(x)
x = Convolution2D(64, (3,3), strides=(1,1), activation='relu')(x)
x = Flatten()(x)
x = Dense(100, activation='relu')(x)
x = Dense(50, activation='relu')(x)
x = Dense(10, activation='relu')(x)
x = keras.layers.concatenate([x, camera_input_bn])
x = Dense(1)(x)

model = Model([inputs, camera_input], outputs=x)
model.compile(loss='mse', optimizer='adam')

In [ ]:
model.summary()

In [14]:
model.fit([X_train, camera_train], y_train, validation_split=0.2, shuffle=True, epochs=7)

model.save(data_dir+'camera_metadata_model.h5')

Train on 38572 samples, validate on 9644 samples
Epoch 1/7
38572/38572 [==============================] - 42s - loss: 0.5708 - val_loss: 0.0145
Epoch 2/7
38572/38572 [==============================] - 42s - loss: 0.0167 - val_loss: 0.0124
Epoch 3/7
38572/38572 [==============================] - 42s - loss: 0.0113 - val_loss: 0.0118
Epoch 4/7
38572/38572 [==============================] - 42s - loss: 0.0092 - val_loss: 0.0115
Epoch 5/7
38572/38572 [==============================] - 42s - loss: 0.0086 - val_loss: 0.0113
Epoch 6/7
38572/38572 [==============================] - 42s - loss: 0.0081 - val_loss: 0.0108
Epoch 7/7
38572/38572 [==============================] - 42s - loss: 0.0079 - val_loss: 0.0131


In [20]:
model.layers[-1].get_weights()


[array([[-0.25042042],
        [-0.06433322],
        [-0.0771196 ],
        [-0.19510733],
        [-0.51878339],
        [-0.50433618],
        [-0.22599629],
        [ 0.39213869],
        [ 0.60182816],
        [ 0.45606974],
        [-0.0352585 ],
        [-0.02821962],
        [-0.0466968 ]], dtype=float32), array([-0.0005411], dtype=float32)]

### NVIDIA Suggested network

In [ ]:
model = Sequential()
model.add(Lambda(lambda x: x - 255.0 / 255.0, input_shape=(160,320,3)))
# model.add(Lambda(lambda x: x - 255.0 / 255.0, input_shape=(66,200,3))) # Checking with NVidia dimensions
model.add(keras.layers.Convolution2D(24, (5,5), strides=(2,2), activation='relu'))
model.add(keras.layers.Convolution2D(36, (5,5), strides=(2,2), activation='relu'))
model.add(keras.layers.Convolution2D(48, (5,5), strides=(2,2), activation='relu'))
model.add(keras.layers.Convolution2D(64, (3,3), strides=(1,1), activation='relu'))
model.add(keras.layers.Convolution2D(64, (3,3), strides=(1,1), activation='relu'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
# model.add(Dense(1, activation='softmax'))

model.compile(loss='mse', optimizer='adam')

In [ ]:
model.summary()

In [ ]:

model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=7)

model.save(data_dir+'nvidia_suggested_model.h5')